# 02. Basic statistics
## Project: Bicycle node network loop analysis

This notebook explores basic statistics of network and loops created in notebook 01, without exploring different scenarios.

Contact: Michael Szell (michael.szell@gmail.com)

Created: 2024-01-29  
Last modified: 2024-10-07

## To do
- [ ] Plot avg/median loop length per node: network, distribution (for all scenarios)
- [ ] Plot distrib of length of loops that have nodes == loop_numnode_bound. To see whether we should increase the bound.
- [ ] Heatmaps over area?

## Parameters

In [ ]:
%run -i setup_parameters.py
load_data = True  # Set to False if data are huge and have already been loaded
debug = True  # Set to True for extra plots and verbosity

## Functions

In [ ]:
%run -i functions.py

## Load data

This can take several minutes.

In [ ]:
if load_data:
    with open(
        PATH["data_out"] + "loopcensus_" + str(LOOP_NUMNODE_BOUND) + ".pkl", "rb"
    ) as f:
        allloops = pickle.load(f)
        alllooplengths = pickle.load(f)
        allloopnumnodes = pickle.load(f)
        allloopmaxslopes = pickle.load(f)
        G = pickle.load(f)
        LOOP_NUMNODE_BOUND = pickle.load(f)
        nodes_id = pickle.load(f)
        nodes_coords = pickle.load(f)
        numloops = pickle.load(f)
        faceloops = pickle.load(f)

In [ ]:
# Plot to double-check
if debug:
    plotCheck(
        G,
        nodes_id,
        nodes_coords,
        vertex_size=get_vertexsize(G.vcount()),
        edge_width=get_edgewidth(G.ecount()),
    )

## Basic statistics: Links, face loops, etc

### Link lengths

In [ ]:
linklengths = [e["weight"] * MPERUNIT for e in G.es]
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0.16, 0.16, 0.76, 0.75])

histxy = axes.hist(
    linklengths, bins=[i * 500 / MPERUNIT for i in list(range(30))], density=False
)
axes.plot([LINK_LIMIT[0], LINK_LIMIT[0]], [0, max(histxy[0])], ":k")
axes.plot([LINK_LIMIT[1], LINK_LIMIT[1]], [0, max(histxy[0])], ":k")
axes.plot([LINK_LIMIT[2], LINK_LIMIT[2]], [0, max(histxy[0])], ":r")
indcond = [
    i for i, x in enumerate(linklengths) if (x >= LINK_LIMIT[0] and x <= LINK_LIMIT[1])
]
massinallowedrange = round(len(indcond) / len(linklengths) * 100)  # Should be high
axes.text(
    (LINK_LIMIT[0] + LINK_LIMIT[1]) / 2,
    max(histxy[0]),
    str(massinallowedrange) + "%",
    horizontalalignment="center",
    verticalalignment="top",
)
axes.text(
    LINK_LIMIT[0] * 0.9,
    max(histxy[0]),
    str(
        round(
            len([i for i, x in enumerate(linklengths) if (x <= LINK_LIMIT[0])])
            / len(linklengths)
            * 100
        )
    )
    + "%",
    horizontalalignment="right",
    verticalalignment="top",
)
axes.text(
    (LINK_LIMIT[1] + LINK_LIMIT[2]) / 2,
    max(histxy[0]),
    str(
        round(
            len(
                [
                    i
                    for i, x in enumerate(linklengths)
                    if (x >= LINK_LIMIT[1] and x <= LINK_LIMIT[2])
                ]
            )
            / len(linklengths)
            * 100
        )
    )
    + "%",
    horizontalalignment="center",
    verticalalignment="top",
)
axes.text(
    LINK_LIMIT[2] * 1.01,
    max(histxy[0]),
    str(
        round(
            len([i for i, x in enumerate(linklengths) if (x > LINK_LIMIT[2])])
            / len(linklengths)
            * 100
        )
    )
    + "%",
    horizontalalignment="left",
    verticalalignment="top",
    color="red",
)

axes.set_xlabel("Length [m]")
axes.set_ylabel("Frequency")
axes.set_title("Link lengths")
axes.set_xlim([0, 1.2 * LINK_LIMIT[2] / MPERUNIT])

fig.savefig(PATH["plot"] + "linklengths")

### Loop lengths and nodes per loop

This can take a few minutes.

In [ ]:
fig = plt.figure(figsize=(8, 3))
axes1 = fig.add_axes([0.1, 0.1, 0.35, 0.8])
axes2 = fig.add_axes([0.55, 0.1, 0.35, 0.8])

axes1.hist(alllooplengths, density=True)
if MPERUNIT == 1000:
    axes1.set_xlabel("Length [km]")
elif MPERUNIT == 1:
    axes1.set_xlabel("Length [m]")
else:
    axes1.set_xlabel("Length")
axes1.set_ylabel("Probability")
axes1.set_title("Loop lengths")

axes2.hist(allloopnumnodes, density=True, bins=list(range(LOOP_NUMNODE_BOUND + 1)))
axes2.set_xlabel("Nodes")
axes2.set_title("Nodes per loop")
axes2.set_xlim([0, LOOP_NUMNODE_BOUND + 0.5])

plt.text(LOOP_NUMNODE_BOUND / 20, 0.01, "Bound: " + str(LOOP_NUMNODE_BOUND))
plt.text(LOOP_NUMNODE_BOUND / 20, 0.04, "Loops: " + str(numloops));

### Face loop lengths and nodes per face loop

In [ ]:
fig = plt.figure(figsize=(8, 3))
axes1 = fig.add_axes([0.08, 0.16, 0.4, 0.75])
axes2 = fig.add_axes([0.58, 0.16, 0.4, 0.75])

facelooplengths = [c["length"] * MPERUNIT for c in faceloops.values()]

histxy = axes1.hist(
    facelooplengths, bins=[i * 1000 / MPERUNIT for i in list(range(50))], density=False
)
if MPERUNIT == 1000:
    axes1.set_xlabel("Length [km]")
elif MPERUNIT == 1:
    axes1.set_xlabel("Length [m]")
else:
    axes1.set_xlabel("Length")
axes1.set_ylabel("Frequency")
axes1.set_title("Face loop lengths")
axes1.plot([FACELOOP_LIMIT[0], FACELOOP_LIMIT[0]], [0, max(histxy[0])], ":k")
axes1.plot([FACELOOP_LIMIT[1], FACELOOP_LIMIT[1]], [0, max(histxy[0])], ":r")
axes1.text(
    (FACELOOP_LIMIT[0] + FACELOOP_LIMIT[1]) / 2,
    max(histxy[0]),
    str(
        round(
            len(
                [
                    i
                    for i, x in enumerate(facelooplengths)
                    if (x >= FACELOOP_LIMIT[0] and x <= FACELOOP_LIMIT[1])
                ]
            )
            / len(facelooplengths)
            * 100
        )
    )
    + "%",
    horizontalalignment="center",
    verticalalignment="top",
)
axes1.text(
    FACELOOP_LIMIT[0] * 0.95,
    max(histxy[0]),
    str(
        round(
            len([i for i, x in enumerate(facelooplengths) if (x < FACELOOP_LIMIT[0])])
            / len(facelooplengths)
            * 100
        )
    )
    + "%",
    horizontalalignment="right",
    verticalalignment="top",
)
axes1.text(
    FACELOOP_LIMIT[1] * 1.01,
    max(histxy[0]),
    str(
        round(
            len([i for i, x in enumerate(facelooplengths) if (x > FACELOOP_LIMIT[1])])
            / len(facelooplengths)
            * 100
        )
    )
    + "%",
    horizontalalignment="left",
    verticalalignment="top",
    color="red",
)
axes1.set_xlim([0, 50000 / MPERUNIT])


axes2.hist(
    [c["numnodes"] for c in faceloops.values()], bins=list(range(30)), density=False
)
axes2.set_xlabel("Nodes")
axes2.set_title("Face loop nodes")
axes2.set_xlim([0, 30])

fig.savefig(PATH["plot"] + "faceloopstats");

### Conforming and non-conforming face loops

In [ ]:
okedges = set()
for c in faceloops.values():
    if (
        c["length"] * MPERUNIT >= FACELOOP_LIMIT[0]
        and c["length"] * MPERUNIT <= FACELOOP_LIMIT[1]
    ):
        okedges = okedges.union(set(c["edges"]))

edge_colors = []
for e in G.es:
    if e.index in okedges:
        edge_colors.append("green")
    else:
        edge_colors.append("grey")

fig = plotCheck(
    G,
    nodes_id,
    nodes_coords,
    vertex_size=get_vertexsize(G.vcount()),
    edge_color=edge_colors,
)
plt.text(0, 0.04, "Conforming face loops highlighted")
plt.tight_layout()
fig.savefig(PATH["plot"] + "faceloops_conforming")

In [ ]:
toosmalledges = set()
toolargeedges = set()
for c in faceloops.values():
    if c["length"] * MPERUNIT > FACELOOP_LIMIT[1]:
        toolargeedges = toolargeedges.union(set(c["edges"]))
    elif c["length"] * MPERUNIT < FACELOOP_LIMIT[0]:
        toosmalledges = toosmalledges.union(set(c["edges"]))

edge_colors = []
for e in G.es:
    if e.index in toolargeedges:
        edge_colors.append("red")
    elif e.index in toosmalledges:
        edge_colors.append("orange")
    else:
        edge_colors.append("grey")

fig = plotCheck(
    G,
    nodes_id,
    nodes_coords,
    vertex_size=get_vertexsize(G.vcount()),
    edge_color=edge_colors,
)
plt.text(0, 0.04, "Non-conforming face loops highlighted")
plt.tight_layout()
fig.savefig(PATH["plot"] + "faceloops_nonconforming")